In [0]:
@logs
def hourly_weather(df):
    from pyspark.sql.functions import hour,col,lit
    from datetime import datetime
    df=df.withColumn('timeid',hour('created_on')).withColumn('Datex',col('created_on').cast('date'))
    try:
        timeid,date=df.select('timeid','Datex').first()
        dateid=str(date).replace('-','')
        query=f"Delete from fact_hourly_weather where time_id='{timeid}' and Date_id='{dateid}';"
        spark.sql(query)
    except:
        pass
    for row in df.rdd.collect():
        date_str = row.date

        # Split the date column into separate date and time components
        date_time = date_str.split(" ")
        date_id = date_time[0].replace("-", "")
        time_id = int(date_time[1].split(":")[0])

        # Perform further processing or store the extracted date_id and time_id as desired
        df = df.withColumn("date_id", lit(date_id))
        df = df.withColumn("time_id",lit(time_id))
        
    date_df=spark.table('dim_date')
    time_df=spark.table('dim_time')

    joined_df = df.join(date_df, df['date_id'] == date_df['Date_id'], 'inner')\
    .join(time_df, df['time_id'] == time_df['time_id'], 'inner')\
    .select(df['date_id'], df['date'], df['dt'], df['time_id'], df['city_id'], df['temp'], df['timezone'], df['temp_min'], df['temp_max'], df['pressure'], df['humidity'], df['visibility'],df['load_run_id'],df['created_on'],df['created_by'])
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    
    return joined_df,start,end


In [0]:
@logs
def daily_weather(df):
    from datetime import datetime
    from pyspark.sql.functions import hour, col, min, max, mean
    df=df.withColumn("Date",col("created_on").cast("date"))
    date_df=spark.table("dim_date")
    joined_df=df.join(date_df,df['date']==date_df['fullDate'],'inner').select(
                        date_df.Date_id,
                        df.city_id,
                        df.dt,
                        df.temp,
                        df.timezone,
                        df.temp,
                        df.temp_min,
                        df.temp_max,
                        df.pressure,
                        df.humidity,
                        df.visibility,
                      )
    daily_df=joined_df.alias('daily_df')
    try:
        date=df.select('Date').first()
        date_id=str(date).replace('-','')
        daily_df = spark.sql(f"select * from fact_daily_weather where Date_id='{date_id}';")\
                        .drop('load_run_id', 'created_on', 'created_by')
        spark.sql(f"delete from fact_daily_weather where Date_id='{date_id}';")
    except:
        pass
    
    fact_df=joined_df.union(daily_df).groupBy(col('city_id'),col('date_id')) \
                .agg(mean('temp').alias('temp'),
                    min('temp_min').alias('temp_min'),
                    max('temp_max').alias('temp_max'),
                    mean('pressure').cast('int').alias('pressure'),
                    mean('timezone').cast('int').alias('timezone'),
                    mean('humidity').cast('int').alias('humidity'),
                    mean('visibility').cast('int').alias('visibility'),
                   )
    start = datetime.fromtimestamp(df.selectExpr("min(dt)").first()[0])
    end = datetime.fromtimestamp(df.selectExpr("max(dt)").first()[0])
    
    return fact_df,start,end
